In [42]:
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [34]:
# %pip install prettytable

In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow import keras
from keras.layers import Dense, Dropout
from tensorflow.keras import layers
from keras.layers import Activation
from keras.models import Sequential

import prettytable
from sklearn import metrics
from math import sqrt

from multiprocessing.pool import ThreadPool

import warnings; warnings.simplefilter('ignore')

In [36]:
def pre_process_data(data,null_threshold):
    """
    Drops Date and Unix Date columns from the data.
    Drops the columns which has null values more than specified null_threshold.
    Replaces infinite values with NAN.
    Drops the rows which has null values.

    Parameters
    ----------
    data : dataframe

    null_threshold : numeric
        numeric value describing the amount of null values that can be present.

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    """
    
    data.drop(columns=['Unix Date','Date'],axis=1,inplace=True)
    total = data.shape[0]
    for col in data.columns:
        if null_threshold * total / 100 < data[col].isnull().sum():
            data.drop(columns=[col],axis=1,inplace=True)
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data = data.apply(pd.to_numeric,errors='coerce')
    data.dropna(axis=0,inplace=True)
    return data

In [37]:
def dependent_column(data,column):
    """
    Removes all the Next Day columns.
    Removes all the non Growth Rate Columns (GR)
    add the predictor column to list of columns.

    Parameters
    ----------
    data : dataframe

    column : string
        name of the predictor column 

    Returns
    -------
    data : dataframe
        an updated dataframe after performing all the opertaions.
    column : string
        name of the predictor column
    """
    cols = [col for col in data.columns if "next" not in col.lower() and col.lower().endswith("gr")]
    cols.append(column)
    data = data[cols]
    return (data,column)

In [38]:
combinations = list()

def get_combinations(arr):
    n = len(arr)
    indices = [0 for i in range(n)]
    while (1):
        res = list()
        for i in range(n):
            res.append(arr[i][indices[i]])
        combinations.append(res)
        next = n - 1
        while (next >= 0 and
              (indices[next] + 1 >= len(arr[next]))):
            next-=1
        if (next < 0):
            return
        indices[next] += 1
        for i in range(next + 1, n):
            indices[i] = 0
u = [25,75,50,100]
b = [25,32,48,64]
e = [25,50,75,100]
arr = [u,b,e]
get_combinations(arr)

In [39]:
def create_model(X_train,layers=3,units = 32):
    model = Sequential()
    model.add(Dense(units=units,input_dim=(X_train.shape[1]),activation = 'relu'))
    model.add(Dense(units=units,activation = 'relu'))
#     model.add(Dropout(0.2))
#     for i in range(layers):
#         model.add(Dense(units=units,activation = 'relu'))
#         model.add(Dropout(0.2))
    model.add(Dense(units = 1,activation = 'relu'))
    return model

In [40]:
def backpropogation(X,Y,layers = 3,epochs = 5, batch_size = 32,units=50):

    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3,random_state = 0)
    model = create_model(x_train,layers,units)
    model.compile(loss="mse", optimizer="Adam", metrics=['accuracy'])
    history = model.fit(x_train, y_train, batch_size=100, epochs=30, validation_data = (x_test, y_test),shuffle = False,verbose=0)
    y_pred = model.predict(x_test)
    
    rmse = sqrt(metrics.mean_squared_error(y_test, y_pred))
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    r2 = metrics.r2_score(y_test, y_pred)

    c = 0
    for a,b in zip(y_pred,y_test):
        if a * b >= 0:
            c += 1
    direction = c/len(y_test)
    
    myres =  {"root_mean_squared_error":rmse,"mean_absolute_error":mae,"mean_squared_error":mse,"rsquared_adj":r2}
    myres.update({"units":units,"epochs":epochs,"batch_size":batch_size,"direction":direction})
    print("done")
    return myres


In [41]:
path = os.getcwd()
path

'C:\\Users\\venu\\Desktop\\Stock Market Analysis'

In [ ]:
%%time
for filename in os.listdir(os.path.join(path,"Data/Stock")):
    if (filename.startswith("gr")):
        df = pd.read_csv(os.path.join(path,"Data\Stock\\" + filename))
        column = "Next Day Close Price GR"
        df = pre_process_data(df,60)
        (df,column) = dependent_column(df,column)
        X = df[df.columns[:-1]]
        Y = df[column]
        arguments = list()
        for units,batch_size,epochs in combinations:
            data = [X,Y,layers,epochs,batch_size,units]
            arguments.append(data)
        threads = ThreadPool(4)
        result = threads.starmap(backpropogation,arguments)
        resultdf = pd.DataFrame(result)
        resultdf.to_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\Models Results\\" + filename[2:8] + "_bpnn_regression"  + ".csv",index=None)

In [17]:
path = os.path.join(os.getcwd(),"Data\\Models Results")
path

'C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\Models Results'

In [21]:
companies = {"500112" : "SBIN" ,
"500325" : "RELIANCE INDUSTRIES LTD",
"532540" : "TATA CONSULTANCY SERVICES LTD" ,
"500209" : "INFOSYS LTD", 
"532174" : "ICICI BANK LTD", 
"507685" : "WIPRO LTD", 
"530965" : "INDIAN OIL CORPORATION LTD", 
"500182" : "HERO MOTOCORP LTD", 
"532210" : "CITY UNION BANK LTD", 
"500180" : "HDFC Bank Ltd",
"500680" : "PFIZER LTD", 
"506395" : "COROMANDEL iNTERNATIONAL LTD",
"500770" : "TATA CHEMICALS LTD", 
"500085" : "CHAMBAL FERTILISERS & CHEMICALS LTD", 
"501425" : "BOMBAY BURMAH TRADING CORP.LTD", 
"532899" : "KAVERI SEED COMPANY LTD", 
"537291" : "NATH BIO-GENES (INDIA) LTD", 
"500790" : "NESTLE INDIA LTD", 
"500825" : "BRITANNIA INDUSTRIES LTD", 
"533155" : "JUBILANT FOODWORKS LTD", 
"533287" : "ZEE LEARN LTD", 
"533260" : "CAREER POINT LTD", 
"539921" : "SHANTI EDUCATIONAL INITIATIVES LTD", 
"542602" : "EMBASSY OFFICE PARKS REIT", 
"543217" : "MINDSPACE BUSINESS PARKS REIT", 
"543261" : "BROOKFIELD INDIA REAL ESTATE TRUST REIT", 
"532538" : "ULTRATECH CEMENT LTD", 
"500387" : "SHREE CEMENT LTD", 
"500425" : "AMBUJA CEMENTS LTD", 
"532689" : "PVR LTD", 
"532706" : "INOX LEISURE LTD", 
"532163" : "SAREGAMA INDIA LTD", 
"524715" : "SUN PHARMACEUTICAL INDUSTRIES LTD", 
"532488" : "DIVI'S LABORATORIES LTD",
"500124" : "DR.REDDY'S LABORATORIES LTD"}

In [ ]:
best_bpnn = pd.DataFrame()
for filename in os.listdir(path):
    if filename.endswith("_bpnn_regression.csv") and "final" not in filename:
        df = pd.read_csv(os.path.join(path,filename))
        table = prettytable.PrettyTable()
        table.field_names = df.columns
        table.title = str(filename[:6]) + "-bpnn_regression"
        for _,row in df.iterrows():
            row = [round(r,6) for r in row]
            table.add_row(row)
        df["Company"] = str(filename[:6]) + "-" + companies(str(filename[:6]))
        print(table)
        best_bpnn = best_bpnn.append(df.sort_values(by=["direction"],ascending=[False]).head(1),ignore_index=True)
best_bpnn.to_csv(os.path.join("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\Models Results\\" + filename[:6] + "final_bpnn_regression"+".csv"),index=None)

In [ ]:
besttable = prettytable.PrettyTable()
besttable.field_names = df.columns
besttable.title = "best" + " bpnn"
for _,row in best_bpnn.iterrows():
    row = [round(r,6) if isinstance(r,(int,float)) else r for r in row]
    besttable.add_row(row)
print(besttable)